# Access to Fors2 Spectra

Compare spectra in observation frame and rest frame

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- creation date : 2024/02/09
- update : 2024/02/09


The purpose is to recalibrate redshift by emission line

The first goal is to find in which frame (rest or observation) the Fors2 spectra are.
This guess is tested against the tabulated resdhift.

Author Sylvie Dagoret-Campagne
Afflilation : IJCLab/IN2P3/CNRS
Organisation : LSST-DESC
creation date : 2024-02-09
last update : 2024-02-09

In [ ]:
import h5py
import pandas as pd
import numpy as np
import os
import re
from astropy.io import fits
from astropy.table import Table
import matplotlib as mpl
import matplotlib.pyplot as plt
#%matplotlib inline
%matplotlib ipympl
import matplotlib.colors as colors
import matplotlib.cm as cmx
import collections
from collections import OrderedDict
import re
import matplotlib.gridspec as gridspec
from sklearn.gaussian_process import GaussianProcessRegressor, kernels


In [ ]:
plt.rcParams["figure.figsize"] = (12,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
from fors2pcigale.fors2starlightio import Fors2DataAcess

In [ ]:
Lyman_lines = [1220., 1030. ,973.,950., 938., 930.]
Balmer_lines = [6562.791,4861.351,4340.4721,4101.740,3970.072,3889.0641,3835.3971]
Paschen_lines = [8750., 12820., 10938.0,10050., 9546.2, 9229.7,9015.3, 8862.89,8750.46,8665.02]
Brackett_lines = [40522.79, 26258.71, 21661.178, 19440., 18179.21]
Pfund_lines = [ 74599.0, 46537.8, 37405.76 , 32969.8, 30400.]
all_Hydrogen_lines = [ Lyman_lines, Balmer_lines, Paschen_lines, Brackett_lines, Pfund_lines]
Color_lines = ["purple", "blue", "green", "red","grey"]
Balmer_thres = 3645.6
Lyman_thres = 911.267
Paschen_thres = 8200.
Brackett_thres = 14580.
Pfund_lines = 22800.
all_Hydrogen_thres = [Lyman_thres , Balmer_thres, Paschen_thres, Brackett_thres, Pfund_lines]


In [ ]:
D4000_red = [4050.,4250] 
D4000_blue = [3750.,3950.]
W_BALMER = [Balmer_thres, Balmer_lines[0]]
W_LYMAN = [Lyman_thres, Lyman_lines[0]]

In [ ]:
def plot_hydrogen_lines(ax):
    nth = len(all_Hydrogen_thres)
    for idx,group_lines in enumerate(all_Hydrogen_lines):
        # select only Lyman and Balmer
        if idx<2:
            color = Color_lines[idx]
            for wl_line in group_lines:
                ax.axvline(wl_line,color=color,lw=0.5)
            if idx< nth:
                ax.axvline(all_Hydrogen_thres[idx],color=color,linestyle=":")
    ax.axvspan(W_LYMAN[0],W_LYMAN[1],facecolor='green', alpha=0.5)
    ax.axvspan(W_BALMER[0],W_BALMER[1],facecolor='yellow', alpha=0.5)

In [ ]:
def plot_hydrogen_lines_redshift(ax,z):
    nth = len(all_Hydrogen_thres)
    for idx,group_lines in enumerate(all_Hydrogen_lines):
        # select only Lyman and Balmer
        if idx<2:
            color = Color_lines[idx]
            for wl_line in group_lines:
                ax.axvline(wl_line*(1+z),color=color,lw=0.5)
            if idx< nth:
                ax.axvline(all_Hydrogen_thres[idx]*(1+z),color=color,linestyle=":")
    ax.axvspan(W_LYMAN[0]*(1+z),W_LYMAN[1]*(1+z),facecolor='green', alpha=0.5)
    ax.axvspan(W_BALMER[0]*(1+z),W_BALMER[1]*(1+z),facecolor='yellow', alpha=0.5)

In [ ]:
fors2 = Fors2DataAcess()

In [ ]:
fors2.plot_allspectra()

In [ ]:
fors2_tags = fors2.get_list_of_groupkeys()
len(fors2_tags)

In [ ]:
list_of_fors2_attributes = fors2.get_list_subgroup_keys()
print(list_of_fors2_attributes)

In [ ]:
bwr_map = plt.get_cmap('bwr')
reversed_map = bwr_map.reversed() 
cNorm = colors.Normalize(0., vmax=1.)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=bwr_map)

In [ ]:
for specname in fors2_tags:
    attr = fors2.getattribdata_fromgroup(specname)
    #print("=======================================================================================")
    #print(attr)
    redshift = attr['redshift']
    lines = attr['lines']
    the_color = scalarMap.to_rgba(redshift, alpha=1)
    the_dict_sed =  fors2.getspectrum_fromgroup(specname)
    wl = the_dict_sed["wl"] 
    fl = the_dict_sed["fl"] 
    ymax = fl.max()
    fig,(ax1,ax2) = plt.subplots(1,2,figsize=(16,4))
    ax1.plot(wl,fl,'-',color=the_color)
    title = f"{specname}, z={redshift:.2f}, lines = {lines}"
    ax1.set_title(title)
    ax1.axvline(wl.min(),color="k")
    ax1.axvline(wl.max(),color="k")
    ax1.set_xlim(4000.,10000.)
    ax1.set_ylim(0.,ymax)
    ax1.set_xlabel("$\lambda (\\AA)$")
    ax1.set_ylabel("$F_\lambda(\lambda) (A.U)$")
    ax1.grid()
    plot_hydrogen_lines_redshift(ax1,redshift)

    wl2 = wl/(1+redshift)
    
    ax2.plot(wl2,fl,'-',color=the_color)
    #title = f"{specname}, z={redshift:.2f}, lines = {lines}"
    #ax2.set_title(title)
    ax2.axvline(wl2.min(),color="k")
    ax2.axvline(wl2.max(),color="k")
    ax2.set_xlim(2000.,8000.)
    if not np.isnan(ymax):
        ax2.set_ylim(0.,ymax)
    ax2.set_xlabel("$\lambda (\\AA)$")
    ax2.set_ylabel("$F_\lambda(\lambda) (A.U)$")
    ax2.grid()
    plot_hydrogen_lines(ax2)

    
    cbar=fig.colorbar(scalarMap , ax=ax2)
    cbar.ax.set_ylabel('redshift')

    plt.show()
    